In [1]:
#!pip install tensorflow
#!pip install keras

<h3>Import Libraries</h3>

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

<h3>EDA</h3>

In [3]:
df1 = pd.read_csv('train.csv')
df1.shape

(891, 12)

In [4]:
df2 = pd.read_csv('test.csv')
df2.shape

(418, 11)

In [5]:
df = df1.append(df2)
df.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
423,424,0.0,3,"Danbom, Mrs. Ernst Gilbert (Anna Sigrid Maria ...",female,28.0,1,1,347080,14.4000,NaN,S
175,176,0.0,3,"Klasen, Mr. Klas Albin",male,18.0,1,1,350404,7.8542,NaN,S
166,1058,NaN,1,"Brandeis, Mr. Emil",male,48.0,0,0,PC 17591,50.4958,B10,C
327,1219,NaN,1,"Rosenshine, Mr. George (Mr George Thorne"")""",male,46.0,0,0,PC 17585,79.2000,NaN,C
200,1092,NaN,3,"Murphy, Miss. Nora",female,NaN,0,0,36568,15.5000,NaN,Q
878,879,0.0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S
127,128,1.0,3,"Madsen, Mr. Fridtjof Arne",male,24.0,0,0,C 17369,7.1417,NaN,S
703,704,0.0,3,"Gallagher, Mr. Martin",male,25.0,0,0,36864,7.7417,NaN,Q
624,625,0.0,3,"Bowen, Mr. David John ""Dai""",male,21.0,0,0,54636,16.1000,NaN,S
682,683,0.0,3,"Olsvigen, Mr. Thor Anderson",male,20.0,0,0,6563,9.2250,NaN,S


<h2>Feature Enginnering</h2>

In [6]:
del df['Survived']

In [7]:
df['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [8]:
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=False)
df['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Major         2
Mlle          2
Jonkheer      1
Don           1
Sir           1
Mme           1
Dona          1
Countess      1
Capt          1
Lady          1
Name: Title, dtype: int64

In [9]:
df['Title'] = df['Title'].replace(['Ms', 'Mlle'], 'Miss')
df['Title'] = df['Title'].replace(['Mme', 'Countess', 'Lady', 'Dona'], 'Mrs')
df['Title'] = df['Title'].replace(['Dr', 'Major', 'Col', 'Sir', 'Rev', 'Jonkheer', 'Capt', 'Don'], 'Mr')

In [10]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df.FamilySize.value_counts()

1     790
2     235
3     159
4      43
6      25
5      22
7      16
11     11
8       8
Name: FamilySize, dtype: int64

In [11]:
df["Age"].fillna(df.groupby("Title")["Age"].transform("median"), inplace= True)

In [12]:
df.groupby("Title")["Age"].transform("median")

0      30.0
1      35.5
2      22.0
3      35.5
4      30.0
       ... 
413    30.0
414    35.5
415    30.0
416    30.0
417     4.0
Name: Age, Length: 1309, dtype: float64

In [13]:
df["Fare"].fillna(df["Fare"].mean(), inplace= True)

In [14]:
df.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
Title             0
FamilySize        0
dtype: int64

In [15]:
df['Embarked'].fillna('S', inplace = True)

In [16]:
del df['Cabin']
del df['Name']

In [17]:
df.drop(['PassengerId', 'Ticket'], axis = 1, inplace = True)

In [18]:
df['Sex'] = df['Sex'].astype('category')

In [19]:
df['Embarked'] = df['Embarked'].astype('category')

In [20]:
df.Title.value_counts()

Mr        783
Miss      264
Mrs       201
Master     61
Name: Title, dtype: int64

In [21]:
df['Title'] = df['Title'].astype('category')

In [22]:
df.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize
402,3,female,21.0,1,0,9.8250,S,Miss,2
107,3,male,30.0,0,0,7.7750,S,Mr,1
127,3,male,24.0,0,0,7.1417,S,Mr,1
353,2,male,49.0,1,2,65.0000,S,Mr,4
266,3,male,16.0,4,1,39.6875,S,Mr,6


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Pclass      1309 non-null   int64   
 1   Sex         1309 non-null   category
 2   Age         1309 non-null   float64 
 3   SibSp       1309 non-null   int64   
 4   Parch       1309 non-null   int64   
 5   Fare        1309 non-null   float64 
 6   Embarked    1309 non-null   category
 7   Title       1309 non-null   category
 8   FamilySize  1309 non-null   int64   
dtypes: category(3), float64(2), int64(4)
memory usage: 75.9 KB


In [24]:
df.isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [25]:
df_dummies = pd.get_dummies(df, drop_first =True)
df_dummies.sample(5)

,Pclass,Age,SibSp,Parch,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs
273,3,22.0,1,0,15.500,2,0,1,0,1,0,0
282,3,16.0,0,0,9.500,1,1,0,1,0,1,0
433,3,17.0,0,0,7.125,1,1,0,1,0,1,0
472,2,33.0,1,2,27.750,4,0,0,1,0,0,1
144,2,18.0,0,0,11.500,1,1,0,1,0,1,0


In [26]:
df_dummies.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
FamilySize    0
Sex_male      0
Embarked_Q    0
Embarked_S    0
Title_Miss    0
Title_Mr      0
Title_Mrs     0
dtype: int64

In [27]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_dummies = sc.fit_transform(df_dummies) #normailize te datas

In [28]:
df1.shape, df2.shape

((891, 12), (418, 11))

In [29]:
n_train = df1.shape[0]
n_test = df2.shape[0]

In [30]:
df_train = df_dummies[:n_train]
df_test = df_dummies[n_train:]

In [31]:
df.shape

(1309, 9)

In [32]:
y = df1['Survived'] #hedef

<h3>Deep Learning</h3>

In [33]:
model = Sequential()
model.add(Dense(9, activation='relu'))
model.add(Dense(9, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid')) #classification 

In [35]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(df_train,y, epochs=650, batch_size=5, verbose=2)

Epoch 1/650
179/179 - 1s - loss: 0.6136 - accuracy: 0.6263
Epoch 2/650
179/179 - 0s - loss: 0.5093 - accuracy: 0.7789
Epoch 3/650
179/179 - 0s - loss: 0.4633 - accuracy: 0.8204
Epoch 4/650
179/179 - 0s - loss: 0.4413 - accuracy: 0.8227
Epoch 5/650
179/179 - 0s - loss: 0.4319 - accuracy: 0.8249
Epoch 6/650
179/179 - 0s - loss: 0.4246 - accuracy: 0.8305
Epoch 7/650
179/179 - 0s - loss: 0.4190 - accuracy: 0.8305
Epoch 8/650
179/179 - 0s - loss: 0.4139 - accuracy: 0.8316
Epoch 9/650
179/179 - 0s - loss: 0.4073 - accuracy: 0.8305
Epoch 10/650
179/179 - 0s - loss: 0.4029 - accuracy: 0.8328
Epoch 11/650
179/179 - 0s - loss: 0.3998 - accuracy: 0.8361
Epoch 12/650
179/179 - 0s - loss: 0.3974 - accuracy: 0.8294
Epoch 13/650
179/179 - 0s - loss: 0.3945 - accuracy: 0.8339
Epoch 14/650
179/179 - 0s - loss: 0.3942 - accuracy: 0.8328
Epoch 15/650
179/179 - 0s - loss: 0.3890 - accuracy: 0.8339
Epoch 16/650
179/179 - 0s - loss: 0.3880 - accuracy: 0.8451
Epoch 17/650
179/179 - 0s - loss: 0.3877 - accura

Epoch 138/650
179/179 - 0s - loss: 0.3360 - accuracy: 0.8496
Epoch 139/650
179/179 - 0s - loss: 0.3353 - accuracy: 0.8563
Epoch 140/650
179/179 - 0s - loss: 0.3349 - accuracy: 0.8597
Epoch 141/650
179/179 - 0s - loss: 0.3344 - accuracy: 0.8519
Epoch 142/650
179/179 - 0s - loss: 0.3345 - accuracy: 0.8631
Epoch 143/650
179/179 - 0s - loss: 0.3333 - accuracy: 0.8552
Epoch 144/650
179/179 - 0s - loss: 0.3389 - accuracy: 0.8519
Epoch 145/650
179/179 - 0s - loss: 0.3346 - accuracy: 0.8586
Epoch 146/650
179/179 - 0s - loss: 0.3353 - accuracy: 0.8631
Epoch 147/650
179/179 - 0s - loss: 0.3345 - accuracy: 0.8530
Epoch 148/650
179/179 - 0s - loss: 0.3330 - accuracy: 0.8631
Epoch 149/650
179/179 - 0s - loss: 0.3328 - accuracy: 0.8563
Epoch 150/650
179/179 - 0s - loss: 0.3364 - accuracy: 0.8608
Epoch 151/650
179/179 - 0s - loss: 0.3333 - accuracy: 0.8586
Epoch 152/650
179/179 - 0s - loss: 0.3323 - accuracy: 0.8608
Epoch 153/650
179/179 - 0s - loss: 0.3362 - accuracy: 0.8597
Epoch 154/650
179/179 - 

Epoch 273/650
179/179 - 0s - loss: 0.3143 - accuracy: 0.8687
Epoch 274/650
179/179 - 0s - loss: 0.3119 - accuracy: 0.8631
Epoch 275/650
179/179 - 0s - loss: 0.3135 - accuracy: 0.8642
Epoch 276/650
179/179 - 0s - loss: 0.3120 - accuracy: 0.8642
Epoch 277/650
179/179 - 0s - loss: 0.3127 - accuracy: 0.8597
Epoch 278/650
179/179 - 0s - loss: 0.3141 - accuracy: 0.8676
Epoch 279/650
179/179 - 0s - loss: 0.3093 - accuracy: 0.8642
Epoch 280/650
179/179 - 0s - loss: 0.3101 - accuracy: 0.8687
Epoch 281/650
179/179 - 0s - loss: 0.3098 - accuracy: 0.8676
Epoch 282/650
179/179 - 0s - loss: 0.3114 - accuracy: 0.8642
Epoch 283/650
179/179 - 0s - loss: 0.3094 - accuracy: 0.8732
Epoch 284/650
179/179 - 0s - loss: 0.3095 - accuracy: 0.8653
Epoch 285/650
179/179 - 0s - loss: 0.3060 - accuracy: 0.8664
Epoch 286/650
179/179 - 0s - loss: 0.3106 - accuracy: 0.8642
Epoch 287/650
179/179 - 0s - loss: 0.3095 - accuracy: 0.8664
Epoch 288/650
179/179 - 0s - loss: 0.3114 - accuracy: 0.8721
Epoch 289/650
179/179 - 

Epoch 408/650
179/179 - 0s - loss: 0.3004 - accuracy: 0.8732
Epoch 409/650
179/179 - 0s - loss: 0.3008 - accuracy: 0.8698
Epoch 410/650
179/179 - 0s - loss: 0.2945 - accuracy: 0.8754
Epoch 411/650
179/179 - 0s - loss: 0.2940 - accuracy: 0.8799
Epoch 412/650
179/179 - 0s - loss: 0.2901 - accuracy: 0.8777
Epoch 413/650
179/179 - 0s - loss: 0.2880 - accuracy: 0.8765
Epoch 414/650
179/179 - 0s - loss: 0.2932 - accuracy: 0.8810
Epoch 415/650
179/179 - 0s - loss: 0.2969 - accuracy: 0.8743
Epoch 416/650
179/179 - 0s - loss: 0.2937 - accuracy: 0.8765
Epoch 417/650
179/179 - 0s - loss: 0.3058 - accuracy: 0.8799
Epoch 418/650
179/179 - 0s - loss: 0.2994 - accuracy: 0.8777
Epoch 419/650
179/179 - 0s - loss: 0.2928 - accuracy: 0.8721
Epoch 420/650
179/179 - 0s - loss: 0.2922 - accuracy: 0.8810
Epoch 421/650
179/179 - 0s - loss: 0.2925 - accuracy: 0.8788
Epoch 422/650
179/179 - 0s - loss: 0.2949 - accuracy: 0.8721
Epoch 423/650
179/179 - 0s - loss: 0.2955 - accuracy: 0.8810
Epoch 424/650
179/179 - 

Epoch 543/650
179/179 - 0s - loss: 0.2807 - accuracy: 0.8844
Epoch 544/650
179/179 - 0s - loss: 0.2860 - accuracy: 0.8799
Epoch 545/650
179/179 - 0s - loss: 0.2863 - accuracy: 0.8799
Epoch 546/650
179/179 - 0s - loss: 0.2865 - accuracy: 0.8810
Epoch 547/650
179/179 - 0s - loss: 0.2831 - accuracy: 0.8777
Epoch 548/650
179/179 - 0s - loss: 0.2866 - accuracy: 0.8810
Epoch 549/650
179/179 - 0s - loss: 0.2902 - accuracy: 0.8765
Epoch 550/650
179/179 - 0s - loss: 0.2901 - accuracy: 0.8799
Epoch 551/650
179/179 - 0s - loss: 0.2906 - accuracy: 0.8743
Epoch 552/650
179/179 - 0s - loss: 0.2860 - accuracy: 0.8799
Epoch 553/650
179/179 - 0s - loss: 0.2898 - accuracy: 0.8810
Epoch 554/650
179/179 - 0s - loss: 0.2855 - accuracy: 0.8754
Epoch 555/650
179/179 - 0s - loss: 0.2875 - accuracy: 0.8777
Epoch 556/650
179/179 - 0s - loss: 0.2823 - accuracy: 0.8822
Epoch 557/650
179/179 - 0s - loss: 0.2861 - accuracy: 0.8799
Epoch 558/650
179/179 - 0s - loss: 0.2835 - accuracy: 0.8799
Epoch 559/650
179/179 - 

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 9)                 117       
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 80        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 338
Trainable params: 338
Non-trainable params: 0
_________________________________________________________________


In [38]:
scores = model.evaluate(df_train,y)

28/28 [==============================] - 0s 443us/step - loss: 0.2654 - accuracy: 0.8878


In [39]:
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 88.78%


<h3>Pred the Model</h3>

In [40]:
prediction = model.predict(df_test)

final = (prediction > 0.5).astype(int).reshape(df_test.shape[0])

output = pd.DataFrame({'PassengerId': df2['PassengerId'], 'Survived' : final})
output.to_csv('predictionDL.csv', index=False)

In [41]:
ctrl = pd.read_csv('predictionDL.csv')
ctrl.sample(30)

,PassengerId,Survived
309,1201,0
303,1195,0
113,1005,1
335,1227,1
217,1109,0
74,966,1
361,1253,1
151,1043,0
298,1190,0
144,1036,1
